# Term2 Sprint17 chapter 3
YOLOv3を使用してシンプソンズデータセットの学習と推定を行う。

## kaggleからシンプソンズデータセットをDLする

In [0]:
!pip install kaggle

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
mkdir .kaggle

In [0]:
ls -a

./  ../  .config/  drive/  .kaggle/  sample_data/


In [0]:
# https://www.kaggle.com/　へアクセスし、ログイン後に右上のプロフィール画像をクリック、さらに"My Account"をクリックする
# このサイトの「API」項目にある"Create New API Token"をクリックすると、kaggle.jsonファイルが自動的にダウンロードされる
# ローカルで、kaggle.json（ダウンロードフォルダにあるはず）をエディターで開く。
# このセルの以下のコードにある token = {'username':'***','key':'***'} における「***」部分を、
# ダウンロードしたkaggle.jsonを参照して書き換え、このセルを実行する

import json

token = {"username":"ishizucat","key":"7ad9397c01d417573ba83f57a190fc46"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [0]:
!chmod 600 /content/.kaggle/kaggle.json      #権限付与

In [0]:
pwd

'/content'

In [0]:
ls -a .kaggle

./  ../  kaggle.json


In [0]:
!cp /content/.kaggle/kaggle.json /root/.kaggle/kaggle.json

In [0]:
# simpsonデータセットのAPIcommandを叩いてデータセットをダウンロードする
# APIcommandはこちらのサイトに。https://www.kaggle.com/alexattia/the-simpsons-characters-dataset

!kaggle datasets download -d alexattia/the-simpsons-characters-dataset

 98% 1.06G/1.08G [00:08<00:00, 168MB/s]
100% 1.08G/1.08G [00:08<00:00, 137MB/s]


In [0]:
#!unzip /content/the-simpsons-characters-dataset.zip  -d Simpsons

In [0]:
# train用のデータセットを解凍

!unzip /content/Simpsons/simpsons_dataset.zip

unzip:  cannot find or open /content/Simpsons/simpsons_dataset.zip, /content/Simpsons/simpsons_dataset.zip.zip or /content/Simpsons/simpsons_dataset.zip.ZIP.


## yolo3のチュートリアルに従って画像および動画を検出してみる

In [0]:
!git clone https://github.com/qqwweee/keras-yolo3.git

Cloning into 'keras-yolo3'...
remote: Enumerating objects: 144, done.
remote: Total 144 (delta 0), reused 0 (delta 0), pack-reused 144
Receiving objects: 100% (144/144), 150.95 KiB | 646.00 KiB/s, done.
Resolving deltas: 100% (65/65), done.


In [0]:
#pre trained model
#https://pjreddie.com/darknet/yolo/
!git clone https://github.com/pjreddie/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 5901, done.
remote: Total 5901 (delta 0), reused 0 (delta 0), pack-reused 5901
Receiving objects: 100% (5901/5901), 6.16 MiB | 7.68 MiB/s, done.
Resolving deltas: 100% (3916/3916), done.


In [0]:
!make

mkdir -p obj
mkdir -p backup
mkdir -p results
gcc -Iinclude/ -Isrc/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -c ./src/gemm.c -o obj/gemm.o
gcc -Iinclude/ -Isrc/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -c ./src/utils.c -o obj/utils.o
gcc -Iinclude/ -Isrc/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -c ./src/cuda.c -o obj/cuda.o
gcc -Iinclude/ -Isrc/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -c ./src/deconvolutional_layer.c -o obj/deconvolutional_layer.o
gcc -Iinclude/ -Isrc/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -c ./src/convolutional_layer.c -o obj/convolutional_layer.o
gcc -Iinclude/ -Isrc/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -c ./src/list.c -o obj/list.o
gcc -Iinclude/ -Isrc/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -c ./src/image.c -o obj/image.o
gcc 

In [0]:
#darknet/cfgに移動して、重みファイルをDL
!wget https://pjreddie.com/media/files/yolov3.weights

--2019-12-18 02:56:09--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  20.5MB/s    in 13s     

2019-12-18 02:56:22 (18.9 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]



In [0]:
#権限付与
!chmod 600 cfg/yolov3.cfg

In [0]:
#自前の写真データで試してみる
!./darknet detect cfg/yolov3.cfg cfg/yolov3.weights data/car_test.jpg

layer     filters    size              input                output
    0 conv     32  3 x 3 / 1   608 x 608 x   3   ->   608 x 608 x  32  0.639 BFLOPs
    1 conv     64  3 x 3 / 2   608 x 608 x  32   ->   304 x 304 x  64  3.407 BFLOPs
    2 conv     32  1 x 1 / 1   304 x 304 x  64   ->   304 x 304 x  32  0.379 BFLOPs
    3 conv     64  3 x 3 / 1   304 x 304 x  32   ->   304 x 304 x  64  3.407 BFLOPs
    4 res    1                 304 x 304 x  64   ->   304 x 304 x  64
    5 conv    128  3 x 3 / 2   304 x 304 x  64   ->   152 x 152 x 128  3.407 BFLOPs
    6 conv     64  1 x 1 / 1   152 x 152 x 128   ->   152 x 152 x  64  0.379 BFLOPs
    7 conv    128  3 x 3 / 1   152 x 152 x  64   ->   152 x 152 x 128  3.407 BFLOPs
    8 res    5                 152 x 152 x 128   ->   152 x 152 x 128
    9 conv     64  1 x 1 / 1   152 x 152 x 128   ->   152 x 152 x  64  0.379 BFLOPs
   10 conv    128  3 x 3 / 1   152 x 152 x  64   ->   152 x 152 x 128  3.407 BFLOPs
   11 res    8                 152 x 

軽自動車を後ろから写した写真に対して、

bus: 78%
car: 68%

という推定結果を得た。

## YOLOにSimpsonsデータセットを学習させる

In [0]:
# keras-yolo3フォルダに移動
cd ../keras-yolo3/

/content/keras-yolo3


In [0]:
#学習ずみの重みをDL
!wget https://pjreddie.com/media/files/yolov3.weights

--2019-12-18 04:45:20--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  19.5MB/s    in 13s     

2019-12-18 04:45:34 (18.5 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]



In [0]:
#モデルをkeras用にコンバート
!python convert.py -w yolov3.cfg yolov3.weights model_data/yolo_weights.h5

Using TensorFlow backend.
Loading weights.
Weights Header:  0 2 0 [32013312]
Parsing Darknet config.
Creating Keras model.


Parsing section net_0
Parsing section convolutional_0
conv2d bn leaky (3, 3, 3, 32)




2019-12-18 04:46:54.833097: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX512F
2019-12-18 04:46:54.852684: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2000179999 Hz
2019-12-18 04:46:54.855712: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x243cbc0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2019-12-18 04:46:54.855745: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2019-12-18 04:46:54.860959: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2019-12-18 04:46:55.033526: I tensorflow/

In [0]:
#classファイルを作成してmodel_dataに設置
#アノテーションファイルを作成してルート直下に配置
#train.py line17付近のパスを修正

!python train.py

Using TensorFlow backend.








2019-12-18 07:22:07.234437: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX512F
2019-12-18 07:22:07.240161: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2000179999 Hz
2019-12-18 07:22:07.240419: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x3052bc0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2019-12-18 07:22:07.240457: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2019-12-18 07:22:07.242675: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2019-12-18 07:22:07.340006: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [0]:
#推定
!python yolo_video.py --image

Using TensorFlow backend.
Image detection mode
 Ignoring remaining command line arguments: ./path2your_video,



2019-12-18 10:44:54.133600: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX512F
2019-12-18 10:44:54.138155: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2000179999 Hz
2019-12-18 10:44:54.138324: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1e0ebc0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2019-12-18 10:44:54.138377: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2019-12-18 10:44:54.140158: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2019-12-18 10:44:54.241477: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative val

Yolov3にシンプソンズデータセットを学習させたが、学習回数が少なかったためか、シンプソンズの写真に対してうまく検出を行うことができなかった。

以上